## Purpose of script:
#### Reviewing Spark's tree-based models implementation
#### Referencing Jose Portilla's "Spark and Python for Big Data with PySpark" course¶

In [6]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
# use .ml.regression for tree-based regression models
from pyspark.ml.classification import (RandomForestClassifier,
                                       DecisionTreeClassifier,
                                       GBTClassifier)

In [2]:
spark = SparkSession.builder.appName('tree_based_models').getOrCreate()

In [3]:
data = spark.read.format('libsvm').load('../Datasets/sample_libsvm_data.txt')

In [4]:
data.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
+-----+--------------------+
only showing top 5 rows



In [5]:
train_data, test_data = data.randomSplit([0.7, 0.3])

In [8]:
dtc = DecisionTreeClassifier()
rfc = RandomForestClassifier(numTrees=100)
gbt = GBTClassifier()

In [9]:
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

In [11]:
dtc_pred = dtc_model.transform(test_data)
rfc_pred = rfc_model.transform(test_data)
gbt_pred = gbt_model.transform(test_data)

In [13]:
dtc_pred.show(1)

+-----+--------------------+-------------+-----------+----------+
|label|            features|rawPrediction|probability|prediction|
+-----+--------------------+-------------+-----------+----------+
|  0.0|(692,[95,96,97,12...|   [28.0,0.0]|  [1.0,0.0]|       0.0|
+-----+--------------------+-------------+-----------+----------+
only showing top 1 row



In [16]:
# using Multiclass as it provide additional accuracy metrics compared to binary
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [17]:
acc_eval = MulticlassClassificationEvaluator(metricName='accuracy')

In [19]:
dtc_acc = acc_eval.evaluate(dtc_pred)
rfc_acc = acc_eval.evaluate(rfc_pred)
gbt_acc = acc_eval.evaluate(gbt_pred)

In [21]:
# this is test data, so accuracy is unusually high
print(f'Decision Tree Model Accuracy: {dtc_acc}')
print(f'Random Forest Model Accuracy: {rfc_acc}')
print(f'Gradient Boost Model Accuracy: {gbt_acc}')

Decision Tree Model Accuracy: 0.8461538461538461
Random Forest Model Accuracy: 1.0
Gradient Boost Model Accuracy: 0.8461538461538461


## Example 2 - College Dataset
#### Classifying colleges as either private or public by its features

In [34]:
data = spark.read.csv('../Datasets/College.csv', inferSchema=True, header=True)

In [35]:
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [36]:
from pyspark.ml.feature import VectorAssembler

In [37]:
print(data.columns)

['School', 'Private', 'Apps', 'Accept', 'Enroll', 'Top10perc', 'Top25perc', 'F_Undergrad', 'P_Undergrad', 'Outstate', 'Room_Board', 'Books', 'Personal', 'PhD', 'Terminal', 'S_F_Ratio', 'perc_alumni', 'Expend', 'Grad_Rate']


In [38]:
input_cols = ['Apps','Accept','Enroll','Top10perc','Top25perc','F_Undergrad','P_Undergrad','Outstate',
    'Room_Board','Books','Personal','PhD','Terminal','S_F_Ratio','perc_alumni','Expend','Grad_Rate']

assembler = VectorAssembler(inputCols=input_cols, outputCol='features')

In [39]:
output = assembler.transform(data)

In [40]:
from pyspark.ml.feature import StringIndexer

In [41]:
indexer = StringIndexer(inputCol='Private', outputCol='PrivateIndex')

In [42]:
output_fixed = indexer.fit(output).transform(output)

In [44]:
final_data = output_fixed.select(['features', 'PrivateIndex'])

In [45]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [46]:
dtc = DecisionTreeClassifier(labelCol='PrivateIndex', featuresCol='features')
rfc = RandomForestClassifier(labelCol='PrivateIndex', featuresCol='features')
gbt = GBTClassifier(labelCol='PrivateIndex', featuresCol='features')

In [47]:
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

In [48]:
dtc_pred = dtc_model.transform(test_data)
rfc_pred = rfc_model.transform(test_data)
gbt_pred = gbt_model.transform(test_data)

In [49]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [50]:
binary_eval = BinaryClassificationEvaluator(labelCol='PrivateIndex')

In [53]:
dtc_auc = binary_eval.evaluate(dtc_pred)
rfc_auc = binary_eval.evaluate(rfc_pred)
gbt_auc = binary_eval.evaluate(gbt_pred)

In [54]:
print(f'Decision Tree Model Area Under Curve: {dtc_auc}')
print(f'Random Forest Model Area Under Curve: {rfc_auc}')
print(f'Gradient Boost Model Area Under Curve: {gbt_auc}')

Decision Tree Model Area Under Curve: 0.9696803196803196
Random Forest Model Area Under Curve: 0.9824175824175824
Gradient Boost Model Area Under Curve: 0.9802197802197802


In [55]:
acc_eval = MulticlassClassificationEvaluator(labelCol='PrivateIndex', metricName='accuracy')

In [57]:
rfc_acc = acc_eval.evaluate(rfc_pred)
print(f'Random Forest Model Accuracy: {rfc_acc}')

Random Forest Model Accuracy: 0.9367088607594937


## Example 3 - Dog Food Spoilage
#### Identifying which chemical is likely linked to dog food spoiling

In [58]:
data = spark.read.csv('../Datasets/dog_food.csv', inferSchema=True, header=True)

In [59]:
data.head(1)

[Row(A=4, B=2, C=12.0, D=3, Spoiled=1.0)]

In [60]:
assembler = VectorAssembler(inputCols=['A','B','C','D'], outputCol='features')

In [62]:
output = assembler.transform(data)

output.head(1)

[Row(A=4, B=2, C=12.0, D=3, Spoiled=1.0, features=DenseVector([4.0, 2.0, 12.0, 3.0]))]

In [63]:
rfc = RandomForestClassifier(labelCol='Spoiled', featuresCol='features')

In [64]:
final_data = output.select(['features', 'Spoiled'])

In [65]:
rfc_model = rfc.fit(final_data)

In [68]:
final_data.head(1)[0][0]

DenseVector([4.0, 2.0, 12.0, 3.0])

In [71]:
# by the ordering of the columns A, B, C, D
# we conclude that C has the highest feature importance
# and therefore is most likely linked to spoilage
rfc_model.featureImportances

SparseVector(4, {0: 0.0233, 1: 0.0151, 2: 0.9365, 3: 0.0251})

In [72]:
spark.stop()